In [26]:
import pandas as pd
import numpy as np
import geopy
import geocoder
import requests
#import googlemaps
import time
import json
import re
from pandasql import sqldf
from geopy.geocoders import GoogleV3
from arcgis.geocoding import reverse_geocode, geocode
from arcgis.geometry import Geometry
from arcgis.gis import GIS
from xml.etree import ElementTree as ET
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [2]:
df = pd.read_excel("Airports_marko.xlsx", index_col=0)

In [29]:
df

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,Postal Code,Only ZIP
0,6523,00A,heliport,Total RF Heliport,40.070985,-74.933689,11.0,NaN,US,US-PA,Bensalem,no,K00A,NaN,00A,https://www.penndot.pa.gov/TravelInPA/airports...,NaN,NaN,{'address': {'Match_addr': 'Total Rf Heliport'...,
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN,{'address': {'Match_addr': 'Aero B Ranch Airpo...,
2,6524,00AK,small_airport,Lowell Field,59.947733,-151.692524,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN,{'address': {'Match_addr': '22000-22514 Sonny ...,99639
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN,{'address': {'Match_addr': 'Epps Airpark Airpo...,
4,506791,00AN,small_airport,Katmai Lodge Airport,59.093287,-156.456699,80.0,NaN,US,US-AK,King Salmon,no,00AN,NaN,00AN,NaN,NaN,NaN,"{'address': {'Match_addr': '99625, Levelock, A...",99625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76415,46378,ZZ-0001,heliport,Sealand Helipad,51.894444,1.482500,40.0,EU,GB,GB-ENG,Sealand,no,NaN,NaN,NaN,http://www.sealandgov.org/,https://en.wikipedia.org/wiki/Principality_of_...,Roughs Tower Helipad,"{'address': {'Match_addr': 'Noordzee', 'LongLa...",
76416,307326,ZZ-0002,small_airport,Glorioso Islands Airstrip,-11.584278,47.296389,11.0,AF,TF,TF-U-A,Grande Glorieuse,no,NaN,NaN,NaN,NaN,NaN,NaN,"{'address': {'Match_addr': 'Indian Ocean', 'Lo...",
76417,346788,ZZ-0003,small_airport,Fainting Goat Airport,32.110587,-97.356312,690.0,NaN,US,US-TX,Blum,no,87TX,NaN,87TX,NaN,NaN,NaN,{'address': {'Match_addr': 'Fainting Goat Airp...,
76418,342102,ZZZW,closed,Scandium City Heliport,69.355287,-138.939310,4.0,NaN,CA,CA-YT,(Old) Scandium City,no,NaN,NaN,NaN,NaN,NaN,"ZZZW, ZZZW, ZYW, YK96","{'address': {'Match_addr': 'Y0B, Yukon', 'Long...",Y0B


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76420 entries, 0 to 76419
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 76420 non-null  int64  
 1   ident              76420 non-null  object 
 2   type               76420 non-null  object 
 3   name               76420 non-null  object 
 4   latitude_deg       76420 non-null  float64
 5   longitude_deg      76420 non-null  float64
 6   elevation_ft       62023 non-null  float64
 7   continent          39425 non-null  object 
 8   iso_country        76161 non-null  object 
 9   iso_region         76420 non-null  object 
 10  municipality       71372 non-null  object 
 11  scheduled_service  76420 non-null  object 
 12  gps_code           41459 non-null  object 
 13  iata_code          8890 non-null   object 
 14  local_code         32806 non-null  object 
 15  home_link          3694 non-null   object 
 16  wikipedia_link     11042 no

In [4]:
df.isnull().sum()

id                       0
ident                    0
type                     0
name                     0
latitude_deg             0
longitude_deg            0
elevation_ft         14397
continent            36995
iso_country            259
iso_region               0
municipality          5048
scheduled_service        0
gps_code             34961
iata_code            67530
local_code           43614
home_link            72726
wikipedia_link       65378
keywords             59361
Postal Code              7
dtype: int64

In [11]:
df['Postal Code'].fillna("No Data", inplace=True)

In [22]:
def str2dict(expression):
    dict_exp = 0
    if expression == "No Data":
        pass
    else:
        dict_exp = eval(expression)['address']['Postal']
    return dict_exp

df["Only ZIP"] = df["Postal Code"].apply(func=str2dict)
#df["Only ZIP"]   

In [47]:
arcgis_api_key = "AAPKc08ca5152a554b42823cc9e4432dd3a53jx6gF63Rh14dnS2NS_RG1hgoL7clphSOOc6b57JcqW2jkXc3MxWlDY-zaMPzN3X"
gis = GIS(url="https://www.arcgis.com", api_key=arcgis_api_key)

def munic_loc(name):

    # Search for the municipality's location
    results = geocode(name)
    if len(results) > 0:
        location = results[0]["location"]
        postcode = reverse_geocode((Geometry({"x":location['x'], "y":location['y'], "spatialReference":{"wkid": 4326}})))
        if postcode['address']['Postal'] == "":
            return "N/A"
        else:
            return postcode['address']['Postal']
    else:
        return "Can't locate municipality"
    
# def rev_geocode_munic_loc(munic_location):

#     if munic_location == "N/A":
#         pass
#     location = reverse_geocode((Geometry({"x":munic_location['x'], "y":munic_location['y'], "spatialReference":{"wkid": 4326}})))
#     if location['address']['Postal'] == "":
#         return "N/A"
#     else:
#         return location['address']['Postal']

In [48]:
df[df["Only ZIP"]==""]["Only ZIP"] = df[df["Only ZIP"]==""]["municipality"].apply(munic_loc)
#df[df["Only ZIP"]==""]["municipality"][0:100].apply(munic_loc)
# for index, row in df[df["Only ZIP"]==""].iterrows():
#     #print(row)
#     loc = munic_loc(row["municipality"])
#     df.at[index, "Only ZIP"] = loc

address should be a string (single line address) or dictionary (with address fields as keys)
address should be a string (single line address) or dictionary (with address fields as keys)
address should be a string (single line address) or dictionary (with address fields as keys)
address should be a string (single line address) or dictionary (with address fields as keys)
address should be a string (single line address) or dictionary (with address fields as keys)
address should be a string (single line address) or dictionary (with address fields as keys)
address should be a string (single line address) or dictionary (with address fields as keys)
address should be a string (single line address) or dictionary (with address fields as keys)
address should be a string (single line address) or dictionary (with address fields as keys)
address should be a string (single line address) or dictionary (with address fields as keys)
address should be a string (single line address) or dictionary (with a

C:\Users\PC\AppData\Local\Temp\ipykernel_20584\3839695063.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df["Only ZIP"]==""]["Only ZIP"] = df[df["Only ZIP"]==""]["municipality"].apply(munic_loc)


In [51]:
df.to_excel("Airports_marko_1.xlsx", sheet_name="Airports LAT-LON-ZIP")

In [50]:
df.isnull().sum()

id                       0
ident                    0
type                     0
name                     0
latitude_deg             0
longitude_deg            0
elevation_ft         14397
continent            36995
iso_country            259
iso_region               0
municipality          5048
scheduled_service        0
gps_code             34961
iata_code            67530
local_code           43614
home_link            72726
wikipedia_link       65378
keywords             59361
Postal Code              0
Only ZIP                 0
dtype: int64

In [39]:
df[df["Only ZIP"]==""]

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,Postal Code,Only ZIP
0,6523,00A,heliport,Total RF Heliport,40.070985,-74.933689,11.0,NaN,US,US-PA,Bensalem,no,K00A,NaN,00A,https://www.penndot.pa.gov/TravelInPA/airports...,NaN,NaN,{'address': {'Match_addr': 'Total Rf Heliport'...,
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN,{'address': {'Match_addr': 'Aero B Ranch Airpo...,
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN,{'address': {'Match_addr': 'Epps Airpark Airpo...,
5,6526,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR,{'address': {'Match_addr': 'Newport Hospital a...,
6,322127,00AS,small_airport,Fulton Airport,34.942803,-97.818019,1100.0,NaN,US,US-OK,Alex,no,00AS,NaN,00AS,NaN,NaN,NaN,"{'address': {'Match_addr': 'Fulton Airport', '...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76414,32753,ZYYY,medium_airport,Shenyang Dongta Airport,41.784401,123.496002,NaN,AS,CN,CN-21,"Dadong, Shenyang",no,ZYYY,NaN,NaN,NaN,NaN,NaN,"{'address': {'Match_addr': '辽宁省沈阳市大东区长安街道', 'L...",
76415,46378,ZZ-0001,heliport,Sealand Helipad,51.894444,1.482500,40.0,EU,GB,GB-ENG,Sealand,no,NaN,NaN,NaN,http://www.sealandgov.org/,https://en.wikipedia.org/wiki/Principality_of_...,Roughs Tower Helipad,"{'address': {'Match_addr': 'Noordzee', 'LongLa...",
76416,307326,ZZ-0002,small_airport,Glorioso Islands Airstrip,-11.584278,47.296389,11.0,AF,TF,TF-U-A,Grande Glorieuse,no,NaN,NaN,NaN,NaN,NaN,NaN,"{'address': {'Match_addr': 'Indian Ocean', 'Lo...",
76417,346788,ZZ-0003,small_airport,Fainting Goat Airport,32.110587,-97.356312,690.0,NaN,US,US-TX,Blum,no,87TX,NaN,87TX,NaN,NaN,NaN,{'address': {'Match_addr': 'Fainting Goat Airp...,
